# Compression

Compression reduces dynamic range by reducing an input signal once a threshold is breached.

Components of compression:

- Level detection – measure amplitude
- Gain computer – apply threshold, ratio, knee
- Smoothing (attack/release) – prevent abrupt changes
- Apply gain – multiply input by computed gain


In [ ]:
import numpy as np
import warnings
from IPython.display import Audio, display
from scipy.io import wavfile
import matplotlib.pyplot as plt

og_len = 5000 # 5 seconds
channels = 2  # Stereo audio
sr = 44100 # stream rate
audio_path = "../../resources/drums1.wav"

warnings.simplefilter("ignore", wavfile.WavFileWarning)
sr_loaded, y = wavfile.read(audio_path)  # y has shape (samples, channels) if stereo

# Handle stereo by taking mean. Simplifies the example greatly.
if y.ndim == 2:
    y = y.mean(axis=1)

# Normalize to -1 to 1 range
waveform = y.T.astype(np.float32) / np.max(np.abs(y))
display(Audio(waveform, rate=sr))

In [ ]:
# Linear scale threshold
# Values above the threshold will be compressed
 # linear scale threshold of 0.5 ~ -6dB
threshold = 0.05

# compression ratio. normal is 4:1 or less
ratio = 4.0

# Sample window for envelope follower
# This is the window size that is used to smooth the rectified input signal
# Smoothing prevents distortion artifacts and jittery changes in gain
window_size = int(0.01 * sr)  # 10ms sampling window

# Rectify inputs
# We need absolute values when we calculate the gain reduction amount in decibels
rectified = np.abs(waveform)

# This convolves the rectified input signal with averaging window to get the envelope
envelope = np.convolve(rectified, np.ones(window_size)/window_size, mode='same')

# Gather signal and calculate the target gain
# If the envelope is above the threshold, reduce the gain according to the ratio
# If the envelope is below the threshold, just return the envelope (no gain change)
gain = np.where(
  envelope > threshold,
  threshold + (envelope - threshold) / ratio,
  envelope
)

# Calculate the gain reduction factor for each sample
gain = gain / (envelope + 1e-8)

# Apply the gain reduction to the original signal
# After this step, the signal has been compressed
compressed_signal = waveform * gain

display(Audio(compressed_signal, rate=sr))

plt.figure(figsize=(12, 6))
plt.plot(waveform[:5000], label="Original waveform")
plt.plot(compressed_signal[:5000], label="Compressed waveform", alpha=0.8)
plt.plot(gain[:5000], label="Gain factor", alpha=0.8)
plt.legend()
plt.show()

# === Print some sample gain values ===
print("Envelope values:", envelope[:20])
print("Gain factors   :", gain[:20])